In [ ]:
# 라이브러리 불러오기
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import TranslationDataset
from torchtext.data import Field


import numpy as np

import random

In [ ]:
# 하이퍼 파라미터 설정
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
HIDDEN_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

In [ ]:
from utils.dataloader import Get_Dataset_and_Field
train_iterator, valid_iterator, test_iterator, SRC, TRG = Get_Dataset_and_Field(data_size = 100000,batch_size = 256)

dataloaders_dict = {"train": train_iterator, "val": valid_iterator}

In [ ]:
# 모델 정의
from utils.transformer import Encoder, Decoder, Transformer

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

# 인코더(encoder)와 디코더(decoder) 객체 선언
enc = Encoder(INPUT_DIM, HIDDEN_DIM, ENC_LAYERS,
              ENC_HEADS, ENC_PF_DIM, ENC_DROPOUT, device)
dec = Decoder(OUTPUT_DIM, HIDDEN_DIM, DEC_LAYERS,
              DEC_HEADS, DEC_PF_DIM, DEC_DROPOUT, device)

# Transformer 객체 선언
model = Transformer(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

In [ ]:
# 가중치 초기화
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)


model.apply(initialize_weights)

In [ ]:
# AdamW optimizer로 학습 최적화
LEARNING_RATE = 0.0005
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

# 뒷 부분의 패딩(padding)에 대해서는 값 무시
criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

In [ ]:
def train_model(model, dataloaders_dict, criterion, optimizer, num_epochs):

    print('-----start-------')
    # ネットワークをGPUへ
    model.to(device)

    # epoch시작
    for epoch in range(num_epochs):

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # 모델을 학습 모드로
            else:
                model.eval()   # 모델을 검증 모드로

            epoch_loss = 0.0  # epoch loss 초기화

            for batch in (dataloaders_dict[phase]):

                src = batch.원문.to(device)  # input
                trg = batch.번역문.to(device)  # output

                # optimizer초기화
                optimizer.zero_grad()

                # 순전파 계산
                with torch.set_grad_enabled(phase == 'train'):

                    # 출력 단어의 마지막 인덱스(<eos>)는 제외
                    # 입력을 할 때는 <sos>부터 시작하도록 처리
                    output, _ = model(src, trg[:, :-1])
                    # output: [배치 크기, trg_len - 1, output_dim]
                    # trg: [배치 크기, trg_len]

                    output_dim = output.shape[-1]
                    output = output.contiguous().view(-1, output_dim)
                    # 출력 단어의 인덱스 0(<sos>)은 제외
                    trg = trg[:, 1:].contiguous().view(-1)

                    # output: [배치 크기 * trg_len - 1, output_dim]
                    # trg: [배치 크기 * trg len - 1]

                    # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
                    loss = criterion(output, trg)

                    # 모델이 학습 모드일때 역전파 계산 및 파라미터 업데이트
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # epoch loss 더하기
                    epoch_loss += loss.item()

            # epoch loss 계산
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f}'.format(epoch+1, num_epochs,
                                                               phase, epoch_loss))

            if phase == 'valid' and epoch_loss < best_loss:

                best_loss = epoch_loss
                torch.save(model.state_dict(),
                           'transformer_korea_to_english.pt')

    return model

In [ ]:
# 모델 학습 
num_epochs = 10
model_trained = train_model(net, dataloaders_dict,
                          criterion, optimizer, num_epochs=num_epochs)